In [38]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()


from nltk.corpus import stopwords


#### Prepare data

In [39]:
df_label = pd.read_parquet("../../data/processed/pyarrow/code_sagsomkostninger.parquet")
danish_stopwords = stopwords.words('danish')
#remove more data
df_label = df_label.loc[~df_label["LABEL_party_who_pays"].isna()]
df_ufr = pd.read_parquet("../../data/processed/pyarrow/UfR_text.parquet")
df = pd.merge(df_label.loc[:,["id_verdict","LABEL_party_who_pays","omkostninger"]],df_ufr)

#### Find criminal cases

In [40]:
import re
def straffesag(s):
    if re.search(("rigsadvokat|anklagemyndighed"),s,re.IGNORECASE):
        return True
    else:
        return False

df_label["straffesag"] = df_label["prosecutor"].apply(straffesag)
df = pd.merge(df,df_label.loc[:,["id_verdict","defendant","prosecutor","straffesag"]], left_on = "id_verdict", right_on = "id_verdict")


#### Produce descriptive cross-table

In [97]:
df.groupby(["straffesag","LABEL_party_who_pays"]).count().to_excel("../../data/tables/label_descriptives.xlsx")

In [99]:
df = df.merge(df_ufr.loc[:,["id_verdict","document_category"]])
df_gb = df.groupby(["document_category","straffesag","LABEL_party_who_pays"]).count()
df_gb.pivot_table(index=["document_category","LABEL_party_who_pays"],columns="straffesag",values="id_verdict").to_excel("../../data/tables/label_distribution_by_court.xlsx")

In [100]:
df_gb

id_verdict  \
document_category    straffesag LABEL_party_who_pays               
Byretterne           False      DEF                           19   
                                NO_COST                       24   
                                PROC                           9   
Højesteret           False      DEF                         1407   
                                NO_COST                     1318   
                                PROC                        3750   
                     True       DEF                         1019   
                                NO_COST                        1   
                                PROC                         840   
Sø- og Handelsretten False      DEF                          320   
                                NO_COST                      208   
                                PROC                         189   
                     True       DEF                            5   
                                PROC                           5   
Vestre Landsret      False      DEF                          715   
                                NO_COST                      367   
                                PROC                         842   
                     True       DEF                           44   
                                NO_COST                        2   
                                PROC                          57   
Østre Landsret       False      DEF                          596   
                                NO_COST                      501   
                                PROC                         798   
                     True       DEF                          445   
                                NO_COST                        2   
                                PROC                         393   

                                                      omkostninger  \
document_category    straffesag LABEL_party_who_pays                 
Byretterne           False      DEF                             19   
                                NO_COST                         24   
                                PROC                             9   
Højesteret           False      DEF                           1407   
                                NO_COST                       1318   
                                PROC                          3750   
                     True       DEF                           1019   
                                NO_COST                          1   
                                PROC                           840   
Sø- og Handelsretten False      DEF                            320   
                                NO_COST                        208   
                                PROC                           189   
                     True       DEF                              5   
                                PROC                             5   
Vestre Landsret      False      DEF                            715   
                                NO_COST                        367   
                                PROC                           842   
                     True       DEF                             44   
                                NO_COST                          2   
                                PROC                            57   
Østre Landsret       False      DEF                            596   
                                NO_COST                        501   
                                PROC                           798   
                     True       DEF                            445   
                                NO_COST                          2   
                                PROC                           393   

                                                      verdict_text  title  \
document_category    straffesag LABEL_party_who_pays                        
Byretterne           False      DE

#### Produce spreadsheet for manual encoding

In [42]:
df_gb = df.groupby(["straffesag","LABEL_party_who_pays"]).apply(lambda x: x.sample(frac=0.05))
df_gb = df_gb.drop(columns=["straffesag","LABEL_party_who_pays"]).reset_index().loc[:,["id_verdict","omkostninger","prosecutor","defendant","straffesag","verdict_text"]]
df_gb = df_gb.sample(frac=1)
df_gb.to_excel("manual_coding_of_labels.xlsx")

#### Evaluate manual encoding

In [52]:
df_encoded = pd.read_csv("../../data/tables/encoded-manual-labels.csv")
df_encoded = df_encoded.merge(df.loc[:,["id_verdict","LABEL_party_who_pays"]])

In [115]:
df_encoded["Right_classification"] = df_encoded["Hvem betaler?"] == df_encoded["LABEL_party_who_pays"]
print(len(df_encoded)-len(df_encoded.loc[df_encoded["Right_classification"]==False]))
print(1-len(df_encoded.loc[df_encoded["Right_classification"]==False])/len(df_encoded))

686
0.9884726224783862


In [ ]:
df_encoded.to_csv("../../data/tables/manual_encoded_labels.csv")

In [114]:
df_all_rulings = df.loc[~((df["straffesag"] == True) & (df["LABEL_party_who_pays"]=="NO_COST"))].reset_index(drop=True)
df_all_rulings.loc[:,["id_verdict","LABEL_party_who_pays"]].to_csv("../../data/tables/classified_rulings.csv")